Copyright (C) 2020-2024 - Raytheon BBN Technologies Corp.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the License for the specific
language governing permissions and limitations under the License.

Distribution Statement "A" (Approved for Public Release,
Distribution Unlimited).

This material is based upon work supported by the Defense
Advanced Research Projects Agency (DARPA) under Contract No.
HR001119C0102.  The opinions, findings, and conclusions stated
herein are those of the authors and do not necessarily reflect
those of DARPA.


# Greenwich Meanie Yearly Analysis on a Specified /24 Destination Subnet

Runs analysis on the Meanine Geolocation data for a given subnet

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import ipaddress
import os
import warnings

# Ignoring warnings, otherwise there are warnings about having too many graphs
warnings.filterwarnings('ignore')

### User Defined Variables

In [ ]:

# Set value to 1 to see intermediate outputs for debugging. 0 otherwise (recommended)
debug = 0

# /24 of the destination addresses to be kept
destination_address_of_interest = '10.1.202'

# Set value to 1 to do geolocation analysis using results generated with 'ipinfo.io'. 0 otherwise (recommended)
# If this is 1, add the path to the geolocations database in the next variable 
analyze_geolocation = 0

# full path and filename that contains prior Geolocation database
geo_location_db_file = './db/full_geolocations_db.csv'

# Set value to 1 to analyze device types from lift. 0 otherwise (recommended)
# If this is 1, add the path to the device types database in the next variable 
analyze_device_types = 0

# full path and filename that contains prior device types database
devices_db_file = './db/full_devices_db.csv'

# Directory where meanie text data is stored
input_dir_path = r'/home/nice-user/Yearly/'

# Directory where meanie results are printed
output_dir_path = r'/home/nice-user/Yearly'

# Set value to 1 to save figures to output_dir_path. 0 otherwise
save_figs = 1

# Years in the data to be analyzed
years_of_interest = [2020, 2021, 2022, 2023]

# Countries user wants to extract Information for
countries_of_interest = ["United Kingdom", "Philippines", "Brazil", "India", "Greece", "United States", "Canada", 
                         "Serbia", "Poland", "India", "Croatia", "Indonesia", "South Africa", "China", "Russia", 
                         "Singapore", "Argentina", "Australia", "Czech Republic", "Bangladesh", "France", "Malaysia", "Turkey"]


### Initialization of Variables and Function Declarations

In [ ]:

# Columns in the meanie text files that have data as hex
hex_columns = ['Source_Address', 'Destination_Address', 'Source_Port', 'Destination_Port', 'UDP_Checksum', 'TTL', 'IPID']

# Columns from meanie text that will need to be formatted as IP addresses
ip_columns = ['Source_Address', 'Destination_Address']

# Columns for computing before and after filtering statistics
stat_columns = ['Source_Address', 'Destination_Address', 'TTL', 'Destination_Port', 'Payload_Length', 'Destination_Slash24']

orig_data = pd.DataFrame()
data_stats = {}

for year in years_of_interest:
    data_stats[year] = {}

if(input_dir_path[len(input_dir_path)-1] != '/'):
    input_dir_path = input_dir_path + '/'
    
if(output_dir_path[len(output_dir_path)-1] != '/'):
    output_dir_path = output_dir_path + '/'

all_dest_addresses = [f'{destination_address_of_interest}.{x}' for x in range(0,256)]

if debug == 1:
    print(input_dir_path)
    print(data_stats)

In [ ]:
# Function that computes Stats on the data before and after filtering

def compute_data_stats(orig_data, years_of_interest, stat_columns, prefix):
    for year in years_of_interest:
        current_year_data = orig_data[orig_data['Year'] == year]
        current_year_unique_months = current_year_data['Month'].nunique()
        if debug == 1:
            print(year, ' unique months: ', current_year_unique_months)
        data_stats[year][f'{prefix}_Packet_Counts'] = len(current_year_data)
        data_stats[year][f'{prefix}_Packet_norm'] = data_stats[year][f'{prefix}_Packet_Counts'] / current_year_unique_months
        for column in stat_columns:
            data_stats[year][f'{prefix}_Unique_{column}_Counts'] = current_year_data[column].nunique()
            data_stats[year][f'{prefix}_Unique_{column}s'] = current_year_data[column].unique()
            data_stats[year][f'{prefix}_Unique_{column}_norm'] = data_stats[year][f'{prefix}_Unique_{column}_Counts'] / current_year_unique_months

### Read / Clean / Format / Merge Data and Compute Filtering Stats

In [ ]:

# values in Input data

# Saddr - Hex
# Daddr - Hex
# Sport - Hex
# Dport - Hex
# Proto – always UDP (17)
# Timestamp
# UdpCksum - Hex
# PayloadLen – the length of the payload, in bytes (the UDP length field, minus 8 for the UDP header itself)
# Payload – Hex
# TTL - a hex
# IPID - a hex

print("Reading Meanie Data")

# list to store files
file_list = []

# Iterate directory and find all compatible files
for file in os.listdir(input_dir_path):
    # check only text files and add them to the list
    if file.endswith('.txt'):
        file_list.append(file)

# if there is 1 or more file type of interest
if len(file_list)>=1:

    for file in range(0, len(file_list)):
        full_file_path = input_dir_path+file_list[file]

        # Read data from the file only if the file is not empty, i.e. file size > 0
        if os.path.getsize(full_file_path) > 0:
            current_csv = pd.read_csv(full_file_path, header=None)
            current_csv['FileName'] = file_list[file]
            orig_data = pd.concat([orig_data, current_csv], ignore_index=True)

else:
    print("No compatible Files in the directory")

if debug == 1:
    display(orig_data)


In [ ]:
# if the data is not empty
if len(orig_data)>=1:

    # Format all the read data
    orig_data = orig_data.rename(columns={0: 'Source_Address', 1:'Destination_Address', 2:'Source_Port', 
                                          3:'Destination_Port', 4:'Protocol', 5:'Timestamp', 
                                          6:'UDP_Checksum', 7:'Payload_Length', 8:'Payload', 9:'TTL', 10:'IPID'})
    
    for hc in hex_columns:
        orig_data[hc] = orig_data[hc].apply(lambda x: int(x, 16))
        
    for ic in ip_columns:
        orig_data[ic] = orig_data[ic].apply(lambda x: str(ipaddress.ip_address(x)))
        
    orig_data[['Prefix', 'Year', 'Month', 'Day', 'Hour', 'Post']] = orig_data['FileName'].str.split('-', expand=True)
    orig_data = orig_data.drop(['Prefix', 'Post'], axis=1)
    orig_data[['Year', 'Month', 'Day', 'Hour']] = orig_data[['Year', 'Month', 'Day', 'Hour']].astype(int)
    orig_data['Destination_Slash24'] = orig_data['Destination_Address'].apply(lambda row: '.'.join(row.split('.')[:3]))

    
    if debug == 1:
        display(orig_data)
    
    # Extract Stats about the full data
    compute_data_stats(orig_data, years_of_interest, stat_columns, 'All')
    
    if debug == 1:
        print('Before Filtering')
        for year in years_of_interest:
            print(f'{year} Num Unique Destination Addresses: ', data_stats[year]['All_Unique_Destination_Address_Counts'])
    
    # Keep a copy of the full data without filtering
    unfiltered_data = orig_data
    
    
    # filter the data by address of interest
    orig_data = orig_data[orig_data["Destination_Address"].isin(all_dest_addresses)]
    # Extract Stats about the filtered data
    compute_data_stats(orig_data, years_of_interest, stat_columns, 'Filtered')
    
    if debug == 1:
        print('\nAfter Filtering')
        for year in years_of_interest:
            print(f'{year} Num Unique Destination Addresses: ', data_stats[year]['Filtered_Unique_Destination_Address_Counts'])
    

### Analysis on data filtering

In [ ]:
 # Unique source counts by year and country
packets_by_year_slash24 = unfiltered_data.groupby(['Year', 'Destination_Slash24'])['Source_Address'].count().reset_index()


fig = plt.figure()
pd.pivot_table(packets_by_year_slash24, values = 'Source_Address', index = ['Destination_Slash24'], columns = ['Year']).plot.bar(figsize=(12,6), title='Greenwich Meanie Packets for each /24 by Year')

plt.xlabel("Destination /24")
plt.ylabel("Packet Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}MeaniePacketsByYearAndSlash24.png')
plt.show()

In [ ]:
filter_stats_df = pd.DataFrame(data_stats).transpose()
if debug == 1:
    display(filter_stats_df)

In [ ]:
# Look at Sources and Packets by Years
fig = plt.figure()
filter_stats_df['All_Unique_Source_Address_Counts'].plot.bar( title='All Unique Source Address Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Source Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_sourceAddresses_yearly.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_Source_Address_Counts'].plot.bar(title=f'{destination_address_of_interest} Unique Source Address Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Source Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_sourceAddresses_yearly_filtered.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_Source_Address_norm'].plot.bar( title=f'{destination_address_of_interest} Yearly Unique Source Address Counts Normalized by Months')
plt.xlabel("Year")
plt.ylabel("Normalized Unique Source Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_sourceAddresses_yearly_filtered_normalized.png')
plt.show()


fig = plt.figure()
filter_stats_df['All_Packet_Counts'].plot.bar( title='All Packet Counts by Year')
plt.xlabel("Year")
plt.ylabel("Packet Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_packets_yearly.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Packet_Counts'].plot.bar(title=f'{destination_address_of_interest} Packet Counts by Year')
plt.xlabel("Year")
plt.ylabel("Packet Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_packets_yearly_filtered.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Packet_norm'].plot.bar( title=f'{destination_address_of_interest} Yearly Unique Packets Counts Normalized by Months')
plt.xlabel("Year")
plt.ylabel("Normalized Packet Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_packets_yearly_filtered_normalized.png')
plt.show()

In [ ]:
# Look at TTLs by Years
fig = plt.figure()
filter_stats_df['All_Unique_TTL_Counts'].plot.bar( title='All TTL Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_TTLs_yearly.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_TTL_Counts'].plot.bar(title=f'{destination_address_of_interest} TTL Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_TTLs_yearly_filtered.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_TTL_norm'].plot.bar(title=f'{destination_address_of_interest} Yearly Unique TTL Counts Normalized by Months')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_TTLs_yearly_filtered_normalized.png')
plt.show()

In [ ]:
# Look at Payload Lengths by Years
fig = plt.figure()
filter_stats_df['All_Unique_Payload_Length_Counts'].plot.bar( title='All Payload Length Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_Payload_Lengths_yearly.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_Payload_Length_Counts'].plot.bar(title=f'{destination_address_of_interest} Payload Length Counts by Year')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_Payload_Lengths_yearly_filtered.png')
plt.show()

fig = plt.figure()
filter_stats_df['Filtered_Unique_Payload_Length_norm'].plot.bar(title=f'{destination_address_of_interest} Yearly Unique Payload Length Counts Normalized by Months')
plt.xlabel("Year")
plt.ylabel("Unique Counts")
if save_figs == 1:
    plt.savefig(f'{output_dir_path}Unique_Payload_Lengths_yearly_filtered_normalized.png')
plt.show()

### TTL Data

In [ ]:
for year in years_of_interest:
    plt.figure(figsize=(8,4))
    current_table = orig_data[orig_data['Year']==year]
    ax = current_table['TTL'].plot(kind='hist', bins = 100, title=f'{year} Distribution of TTLs in {destination_address_of_interest}')
    ax.set_xlabel("TTLs")
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}TTL_distribution_{year}.png')

### Payload Length Data

In [ ]:
for year in years_of_interest:
    plt.figure(figsize=(8,4))
    current_table = orig_data[orig_data['Year']==year]
    vc = current_table['Payload_Length'].value_counts(sort = False)
    ax = vc.plot(kind='bar', title=f'{year} Distribution of Payload Lengths in {destination_address_of_interest}', align='edge', width=1.0)
    ax.set_xlabel("Payload Lengths")
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}Payload_Length_distribution_{year}.png')

### Yearly Geolocation Analysis

In [ ]:
if analyze_geolocation == 1:
    continent_coordinates = {'World': {'xlim':[-180, 180], 'ylim':[-90, 90]},
                             'Europe': {'xlim':[-20, 50], 'ylim':[25, 70]},
                             'Asia': {'xlim':[60, 145], 'ylim':[-10, 50]},
                             'North America': {'xlim':[-130, -60], 'ylim':[15, 60]},
                             'South America': {'xlim':[-85, -30], 'ylim':[-55, 15]},
                             'Africa': {'xlim':[-35, 60], 'ylim':[-40,40]},
                             'Australia': {'xlim':[110, 187], 'ylim':[-50, -5]}
    }

In [ ]:
if analyze_geolocation == 1:
    print("Merging with GeoLocations data")

    database_geo_locations = pd.read_csv(geo_location_db_file)
    database_geo_locations = database_geo_locations.rename(columns={'IP': 'Source_Address'})

    if debug == 1:
        display(database_geo_locations)
        database_geo_locations.info()

In [ ]:
# Merge the two tables 
if analyze_geolocation == 1:
    merged_table = pd.merge(orig_data, database_geo_locations, on="Source_Address", how = 'left')

    if debug == 1:
        display(merged_table)


In [ ]:
if analyze_geolocation == 1:
    if debug == 1:
        merged_table.info()
        merged_table['Country'].value_counts()

In [ ]:
if analyze_geolocation == 1:
    # Delete unnecessary columns
    subset_table = merged_table.drop(['Timestamp_y', 'Timestamp_y', 'Payload'], axis=1)

    if debug == 1:
        print('subset_table.size', subset_table.size)
        subset_table.head()

In [ ]:
if analyze_geolocation == 1:
    # Remove duplicate rows so each source exists only once for each year and location 
    subset_table = subset_table.drop_duplicates()

    if debug == 1:
        print('subset_table.size', subset_table.size)
        display(subset_table)

In [ ]:
if analyze_geolocation == 1:
    unique_source_counts_by_year_location = subset_table.groupby(['Year', 'Country', 'Region', 'City', 'Latitude', 'Longitude'])['Source_Address'].nunique().reset_index()
    unique_source_counts_by_year_location.to_csv(f'{output_dir_path}CityCountsByYear.csv', index = False)

In [ ]:
if analyze_geolocation == 1:
    unique_years = unique_source_counts_by_year_location['Year'].unique()
    max_count = unique_source_counts_by_year_location['Source_Address'].max()

    continents = list(continent_coordinates.keys())
    for current_year in unique_years:
        current_year_df = unique_source_counts_by_year_location.loc[unique_source_counts_by_year_location['Year']==current_year]

        for continent in continents:

            fig, ax = plt.subplots(figsize=(12,6))

            # Plotting world map
            worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
            worldmap.plot(color="lightgrey", ax=ax)
            ax.set_axis_off()

            longitudes = current_year_df['Longitude']
            latitudes = current_year_df['Latitude']
            counts = current_year_df['Source_Address']
            # Plotting longitudes and latitudes
            plt.scatter(longitudes, latitudes, s=counts, c=counts, alpha=0.6, vmin=0, vmax=max_count, cmap='autumn')
            plt.colorbar(label="Count of Unique Source Addresses by City")
            plt.xlim(continent_coordinates[continent]['xlim'])
            plt.ylim(continent_coordinates[continent]['ylim'])
            plt.title(f'{continent} {current_year}')
            if save_figs == 1:
                plt.savefig(f'{output_dir_path}{continent}_map_{current_year}.png')
            plt.show()

In [ ]:
# This code is the same chart as above, but each city is normalized by % of global traffic. 
# The values are so small that these graphs are not super useful. 
# Left code in here in case this useful for another set of data, but the figures are not saved to output directory
if analyze_geolocation == 1:

    unique_years = unique_source_counts_by_year_location['Year'].unique()

    continents = list(continent_coordinates.keys())
    for current_year in unique_years:
        current_year_df = unique_source_counts_by_year_location.loc[unique_source_counts_by_year_location['Year']==current_year]
        total_yearly_count = current_year_df['Source_Address'].sum()
        current_year_df['Source_Address'] = current_year_df['Source_Address'].div(total_yearly_count)
        max_count = current_year_df['Source_Address'].max()

        for continent in continents:

            fig, ax = plt.subplots(figsize=(12,6))

            # Plotting world map
            worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
            worldmap.plot(color="lightgrey", ax=ax)
            ax.set_axis_off()

            longitudes = current_year_df['Longitude']
            latitudes = current_year_df['Latitude']
            counts = current_year_df['Source_Address']
            # Plotting longitudes and latitudes
            plt.scatter(longitudes, latitudes, s=counts, c=counts, alpha=0.6, vmin=0, vmax=max_count, cmap='autumn')
            plt.colorbar(label="Annual Normalized Unique Source Addresses by City")
            plt.xlim(continent_coordinates[continent]['xlim'])
            plt.ylim(continent_coordinates[continent]['ylim'])
            plt.title(f'{continent} {current_year}')
            plt.show()

### Country Data

In [ ]:
if analyze_geolocation == 1:
    # Unique source counts by year and country
    unique_sources_by_year_country = subset_table.groupby(['Year', 'Country'])['Source_Address'].nunique().reset_index()

    print('Number of unique countries = ', len(unique_sources_by_year_country['Country'].unique()))
    unique_sources_by_year_country.to_csv(f'{output_dir_path}YearlyCountrySources.csv', index = False)

    unique_sources_by_year_country = unique_sources_by_year_country.loc[unique_sources_by_year_country['Country'].isin(countries_of_interest)]


    pd.pivot_table(unique_sources_by_year_country, values = 'Source_Address', index = ['Country'], columns = ['Year']).plot.bar(figsize=(18,6), title='Yearly Unique Source Addresses by Country')

    if save_figs == 1:
        plt.savefig(f'{output_dir_path}Unique_Country_SourceAddresses.png')


In [ ]:
if analyze_geolocation == 1:
    
    # Unique source counts by year and country - scaled by that year
    unique_sources_by_year_country = subset_table.groupby(['Year', 'Country'])['Source_Address'].nunique().reset_index()

    scaled_sources_by_year_country = pd.DataFrame()

    for year in years_of_interest:
        current_year_df = unique_sources_by_year_country[unique_sources_by_year_country['Year']==year]
        total_yearly_count = current_year_df['Source_Address'].sum()
        current_year_df['Source_Address'] = current_year_df['Source_Address']/total_yearly_count*100
        scaled_sources_by_year_country = pd.concat([scaled_sources_by_year_country, current_year_df])


    scaled_sources_by_year_country = scaled_sources_by_year_country.loc[scaled_sources_by_year_country['Country'].isin(countries_of_interest)]

    pd.pivot_table(scaled_sources_by_year_country, values = 'Source_Address', index = ['Country'], columns = ['Year']).plot.bar(figsize=(18,6), title='Percentage Unique Source Addresses by Country')

    if save_figs == 1:
        plt.savefig(f'{output_dir_path}Unique_Country_SourceAddresses_Percents.png')


In [ ]:
if analyze_geolocation == 1:
    
    # Find Top Countries by Number and Precentage of Sources
    unique_sources_by_year_country = subset_table.groupby(['Year', 'Country'])['Source_Address'].nunique().reset_index()

    top_num_countries = pd.DataFrame()
    top_percent_countries = pd.DataFrame()

    for year in years_of_interest:
        current_year_df = unique_sources_by_year_country[unique_sources_by_year_country['Year']==year]
        top_10_num_sources = current_year_df[['Year', 'Country', 'Source_Address']].sort_values('Source_Address', ascending=False).nlargest(10, 'Source_Address')
        top_num_countries = pd.concat([top_num_countries, top_10_num_sources])
        display(top_10_num_sources)
    #     top_10_num_sources.head(10)

        total_yearly_count = current_year_df['Source_Address'].sum()
        current_year_df['Source_Address'] = current_year_df['Source_Address']/total_yearly_count*100
        top_10_percent_sources = current_year_df[['Year', 'Country', 'Source_Address']].sort_values('Source_Address', ascending=False).nlargest(10, 'Source_Address')
        top_percent_countries = pd.concat([top_percent_countries, top_10_percent_sources])
        display(top_10_percent_sources)


    with pd.ExcelWriter(f'{output_dir_path}Top10Countries.xlsx') as writer:
        top_num_countries.to_excel(writer, sheet_name='Counts', index = False)
        top_percent_countries.to_excel(writer, sheet_name='Percentages', index = False)

In [ ]:
if analyze_geolocation == 1:
    print('Top countries By Unique Source Addresses')
    display(top_num_countries)
    print('Top countries By Unique Source Address Percentages')
    display(top_percent_countries)

In [ ]:
if analyze_geolocation == 1:
    unique_cities_by_country = subset_table.groupby(['Country'])['City'].nunique().reset_index()

    unique_cities_by_country = unique_cities_by_country.loc[unique_cities_by_country['Country'].isin(countries_of_interest)].set_index('Country')

    unique_cities_by_country = unique_cities_by_country.rename(columns={'City': 'Number of Cities'})

    unique_cities_by_country.plot.bar(figsize=(18,6), title='Number of Cities in Each Country')

    if save_figs == 1:
        plt.savefig(f'{output_dir_path}Unique_Country_Cities.png')


### City Data

In [ ]:
if analyze_geolocation == 1:
    
    # Find Top Cities by Number and Precentage of Sources
    unique_sources_by_year_city = subset_table.groupby(['Year', 'Country', 'City'])['Source_Address'].nunique().reset_index()
    num_top_cities = 15

    top_num_cities = pd.DataFrame()
    top_percent_cities = pd.DataFrame()

    for year in years_of_interest:
        current_year_df = unique_sources_by_year_city[unique_sources_by_year_city['Year']==year]
        top_10_num_sources = current_year_df[['Year', 'Country', 'City', 'Source_Address']].sort_values('Source_Address', ascending=False).nlargest(num_top_cities, 'Source_Address')
        top_num_cities = pd.concat([top_num_cities, top_10_num_sources])
        print(f'{year} Top countries By Unique Source Addresses')
        display(top_10_num_sources)

        total_yearly_count = current_year_df['Source_Address'].sum()
        current_year_df['Source_Address'] = current_year_df['Source_Address']/total_yearly_count*100
        top_10_percent_sources = current_year_df[['Year', 'Country', 'City', 'Source_Address']].sort_values('Source_Address', ascending=False).nlargest(num_top_cities, 'Source_Address')
        top_percent_cities = pd.concat([top_percent_cities, top_10_percent_sources])
        print(f'{year} Top countries By Unique Source Address Percentages')
        display(top_10_percent_sources)


    with pd.ExcelWriter(f'{output_dir_path}Top{num_top_cities}Cities.xlsx') as writer:
        top_num_cities.to_excel(writer, sheet_name='Counts', index = False)
        top_percent_cities.to_excel(writer, sheet_name='Percentages', index = False)

### Merge the Devices Data

In [ ]:
devices_merged_table = []

if analyze_device_types == 1:
    print('Reading and combining the device types data')

    devices_df = pd.read_csv(devices_db_file)
    devices_df.drop(['Original_Device_Info', 'Timestamp', 'Device_Info'], axis=1, inplace=True)
    devices_df = devices_df.rename(columns={'IP': 'Source_Address'})

    if debug == 1:
        display(devices_df)

    if analyze_geolocation == 1:
        devices_merged_table = pd.merge(subset_table, devices_df, on="Source_Address", how = 'inner')
    else:
        devices_merged_table = pd.merge(orig_data, devices_df, on="Source_Address", how = 'inner')

    if debug == 1:
        display(devices_merged_table)

In [ ]:
if analyze_device_types == 1:

    def gen_colors(df):
        col_d = {'Server': 'maroon', 'Router': 'blue', 'Recorder': 'black', 'Camera': 'pink', 'NVR': 'green', 
                 'HVR': 'yellow', 'DVR': 'orange'}
        return [col_d[col] for col in df.columns]
    
    devices_by_year = devices_merged_table.groupby(['Year', 'Device_Type'])['Source_Address'].nunique().reset_index()
    
    pivot_data = pd.pivot_table(devices_by_year, values = 'Source_Address', index = ['Year'], columns = ['Device_Type'])
    pivot_data.plot(kind = "bar", stacked = True, color=gen_colors(pivot_data), figsize=(18,6), 
                    grid = True, title=f'{year} Device Types by Years')
    
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}Yearly_devices.png')
    

In [ ]:

# Generate devices plots for all countries
if (analyze_device_types == 1) and (analyze_geolocation == 1):

    devices_by_country = devices_merged_table.groupby(['Year', 'Country', 'Device_Type'])['Source_Address'].nunique().reset_index()
    max_address_count = devices_by_country['Source_Address'].max()
    max_address_count += 5
    
    # Unique source counts by year and country
    for year in years_of_interest:
        current_table = devices_by_country[devices_by_country['Year']==year]

        pivot_data = pd.pivot_table(current_table, values = 'Source_Address', index = ['Country'], columns = ['Device_Type'])
        pivot_data.plot(kind = "bar", stacked = True, color=gen_colors(pivot_data), figsize=(18,6), 
                        ylim=(0, max_address_count), grid = True, title=f'{year} Device Types for All Countries')
#         plt.set_ylim(0, max_address_count)
        
        if save_figs == 1:
            plt.savefig(f'{output_dir_path}Country_devices_{year}.png')


In [ ]:

# Generate devices plots for only the countries of intrest

if (analyze_device_types == 1) and (analyze_geolocation == 1):
    # Unique source counts by year and country
    
    devices_by_country = devices_by_country.loc[devices_by_country['Country'].isin(countries_of_interest)]
    max_address_count = devices_by_country['Source_Address'].max()
    max_address_count += 5
        
    for year in years_of_interest:
        current_table = devices_by_country[devices_by_country['Year']==year]

        pivot_data = pd.pivot_table(current_table, values = 'Source_Address', index = ['Country'], columns = ['Device_Type'])
        pivot_data.plot(kind = "bar", stacked = True, color=gen_colors(pivot_data), figsize=(18,6),
                        ylim=(0, max_address_count), grid = True, title=f'{year} Device Types for Countries of Interest')
        
        plt.tight_layout()
        
        if save_figs == 1:
            plt.savefig(f'{output_dir_path}CountryOfInterest_devices_{year}.png')

